In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 3.0MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/Mens_Shoe_Prices.csv', low_memory=False)

In [0]:
df_usd = df[df.prices_currency == 'USD'].copy()

In [0]:
df_usd['prices_amountmin'] = df_usd.prices_amountmin.astype(np.float)

In [0]:
filter_max = np.percentile(df_usd.prices_amountmin, 99)

In [0]:
df = df_usd[df_usd['prices_amountmin'] < filter_max]

In [10]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
df['manufacturer_cat'] = df['manufacturer'].factorize()[0]
df['colors_cat'] = df['colors'].factorize()[0]
df['name_cat'] = df['name'].factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [0]:
feats = ['brand_cat', 'manufacturer_cat', 'colors_cat', 'name_cat']

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=15, random_state=2020)):
  X = df[feats].values
  y = df['prices_amountmin'].values
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [13]:
run_model(feats)

(-62.52181962994441, 6.706295378419814)

In [0]:
model = RandomForestRegressor (max_depth=10, n_estimators=100, random_state=2020)

In [15]:
run_model(feats, model)

(-54.18731510140486, 3.6048797580625895)

In [16]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))

  for feat in features:
    key = feat['key'].lower().strip()
    value = feat['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

df['features_passed'] = df['features'].map(parse_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [17]:
keys = set()
df['features_passed'].map(lambda x: keys.update(x.keys()))

0        None
1        None
2        None
3        None
4        None
         ... 
19382    None
19383    None
19384    None
19385    None
19386    None
Name: features_passed, Length: 18280, dtype: object

In [18]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_passed.map(lambda feats: feats[key] if key in feats else np.nan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_shipping weight', 'feat_case diameter', 'feat_diamond color',
       'feat_crown', 'feat_bridge width', 'feat_use', 'feat_feature',
       'feat_antiscratch lens coating', 'feat_insulated', 'feat_vehicle type'],
      dtype='object', length=529)

In [0]:
!git add matrix_one/day5.ipynb

In [0]:
!git config --global user.email "m.zajac1988@gmail.com"
!git config --global user.name "Mateusz"

In [159]:
!git commit -m 'Features selection'


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@e1de47ee4724.(none)')


In [33]:
!git push -u origin master

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 4.65 KiB | 952.00 KiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/mattzajac/dw_matrix.git
   9d3c384..a358177  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.


In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [27]:
{k:v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [68]:
#df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
#df['feat_color_cat'] = df['feat_color'].factorize()[0]
#df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
#df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
#df['feat_material_cat'] = df['feat_material'].factorize()[0]

#df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
#df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_material_cat', 

 'feat_fabric content_cat',
 'feat_band width_cat',
 'feat_adjustable_cat'
]

In [0]:
feats_cat = [ x for x in df.columns if '_cat' in x ]

In [0]:
model = RandomForestRegressor (max_depth=5, n_estimators=100, random_state=2020)

In [0]:
result = run_model(feats, model)

In [0]:
  X = df[feats].values
  y = df['prices_amountmin'].values

In [147]:
model = RandomForestRegressor (max_depth=5, n_estimators=100, random_state=2020)
model.fit(X, y)

print(result)
perm = PermutationImportance(model, random_state=2020).fit(X, y)
eli5.show_weights(perm, feature_names = feats)

(-57.27605185600065, 4.00210390445246)


Weight,Feature
0.2643 ± 0.0080,brand_cat
0.1169 ± 0.0135,feat_material_cat
0.0352 ± 0.0060,feat_adjustable_cat
0.0265 ± 0.0037,feat_brand_cat
0.0208 ± 0.0023,feat_band width_cat
0.0172 ± 0.0023,feat_fabric content_cat
